In [1]:
from iotbx.map_manager import map_manager as MapManager
from mmtbx.model import manager as ModelManager
from iotbx.data_manager import DataManager
from iotbx.map_model_manager import map_model_manager as MapModelManager

import os
import pickle
import inspect

In [9]:
from libtbx.test_utils import approx_equal

In [2]:
from phenix.ligands_cc.ligands_cc import LigandsCC, LigandSelection

In [10]:
model_file = "../data/6qi8_4552/6qi8_4552.cif"
map_file = "../data/6qi8_4552/6qi8_4552.map"
assert(os.path.exists(model_file))
assert(os.path.exists(map_file))
dm = DataManager()
dm.process_model_file(model_file)
dm.process_real_map_file(map_file)
model = dm.get_model()

# this is necessary for cryo-em models without crystal symmetry
if model.crystal_symmetry() is None:
    from cctbx.maptbx.box import shift_and_box_model
    model = shift_and_box_model(model)

mmm = MapModelManager(map_manager=dm.get_real_map(), model=dm.get_model())

ligands_cc =  LigandsCC(mmm,resolution=3.5)
ligands_cc.validate()
assert(approx_equal(ligands_cc.ccmask_full_model,0.022970855944436454))



In [11]:
ligands_cc.process_ligands()

In [16]:
assert(len(ligands_cc.ligand_map_model_managers)==6)

expected_ligand_ccmask = [
    -0.080566400250404,
     -0.05039665095274361,
     0.02568153796222316,
     -0.058698045705798986,
     0.10242327846306232,
     -0.07061912170822986]


In [ ]:
for ccmask in expected_ligand_ccmask:
    assert(approx_equal(ccmask,ligands_cc.cc))

In [12]:
ligands_cc.ccmask_ligands

[-0.080566400250404,
 -0.05039665095274361,
 0.02568153796222316,
 -0.058698045705798986,
 0.10242327846306232,
 -0.07061912170822986]

## Get ligands out of a model

In [12]:
mmm = dm.get_map_model_manager()

In [13]:
ligands_cc = LigandsCC(mmm,resolution=3.4)

In [14]:
ligands_cc.ccmask_full_model

In [15]:
ligands_cc.process_ligands()

In [19]:
os.getcwd()

'/home/user/Dropbox/Phenix Start/CryoEM-ReRefine-Ligands/Notebooks'

In [24]:
# write ouput
output_dir = "./"

for mmmlig in ligands_cc.ligand_map_model_managers:
    ligand_name = LigandsCC.ligand_name(mmmlig.model())
    ligand_model_file = os.path.join(output_dir,ligand_name)
    ligand_map_file = os.path.join(output_dir,ligand_name+".map")
    
    dm.write_model_file(mmmlig.model(),
                          filename=ligand_model_file,
                          extension="cif",
                          overwrite=True)
    
    dm.write_real_map_file(mmmlig.map_manager(),
                          filename=ligand_map_file,
                          overwrite=True)

In [25]:
ligands_cc.validate()   

NameError: name 'mmmlig' is not defined

## Get ligand density maps and calculate CCmask

In [9]:
map_manager = dm.get_real_map()
map_manager.set_resolution(entry.resolution)
map_model_manager = MapModelManager(map_manager=map_manager,model=dm.get_model())

ligand_dict, ccmask_values = ligands_cc_from_mmm(map_model_manager)

Found 3 ligands in the model 


In [10]:
print(ccmask_values)

[0.820752074164057, 0.8099958071989116, 0.8403064511105527]


## Optionally write to disk

In [12]:
ligand_dict, ccmask_values = ligands_cc_from_mmm(map_model_manager,
                                                 output_directory="../data/")

Found 3 ligands in the model 
Writing model file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_D.map
Writing map file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_D.cif
Writing model file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_F.map
Writing map file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_F.cif
Writing model file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_B.map
Writing map file: /net/cci-filer3/home/cschlick/CryoEM-ReRefine-Ligands/data/ADP_B.cif
